In [1]:
# %pip install bertopic
# %pip install indicnlp-library
# %pip install transformers


In [2]:
# get processed data

import os
import pandas as pd

file_path = "../data/"

data = pd.read_csv(file_path+"train.csv")



In [3]:
# Convert to list
df=data["news_title"]

docs = [str(i) for i in df.values]


In [4]:
import bpemb
from bpemb import BPEmb
import numpy as np

# Load BPEmb model for Tamil
bpemb_tam = BPEmb(lang="ta", dim=300, vs=200000)

# Generate embeddings for news_title column
embeds = []
for title in docs:
    embed = bpemb_tam.embed(title)
    embeds.append(np.mean(embed, axis=0))
embeds = np.array(embeds)


In [5]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=3, n_components=5, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        prediction_data=True, gen_min_span_tree=True)

/Users/samyuktha/opt/anaconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Tokenize docs trivially (split on spaces)

from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def tokenize_ta(text,return_tensors="pt",*args,**kwargs):
    return indic_tokenize.trivial_tokenize(text)

In [7]:
# Common list of stopwords

stopwords=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']

from sklearn.feature_extraction.text import CountVectorizer

# Create a vectorizer object to generate term document counts for topic representation - TOKENIZATION STEP

vectorizer_model = CountVectorizer(
    stop_words=stopwords,analyzer='word',
    tokenizer=tokenize_ta
)

In [8]:

# Create a BERTopic model

from bertopic import BERTopic

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    verbose=True,
    calculate_probabilities=False,
    embedding_model=bpemb_tam,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    
)




In [9]:
# Fit the model on the documents


topics = topic_model.fit_transform(docs,embeds)



OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2023-03-23 17:37:42,341 - BERTopic - Reduced dimensionality
2023-03-23 17:37:49,838 - BERTopic - Clustered reduced embeddings


In [10]:
# Get the topics

topic_model.get_topic_info()



,Topic,Count,Name
0,-1,70240,-1_கைது_அரசு_பரபரப்பு_கொலை
1,0,3232,0_பலி_மோதி_லாரி_பரிதாப
2,1,3063,1_டெஸ்ட்_இந்தியா_வெற்றி_போட்டி
3,2,2732,2_மக்கள்_மறியல்_குடிநீர்_சாலை
4,3,2385,3_கோயிலில்_திருவிழா_பக்தர்கள்_கோயில்
...,...,...,...
176,175,84,175_டும்_வந்தாச்சு_‘குடிமகன்_‘மங்களம்
177,176,81,176_மீனவர்கள்_ஸ்டிரைக்_ராமேஸ்வரம்_இலங்கை
178,177,81,177_கட்சி_ஜனதா_தலைவர்_காங்கிரஸ்
179,178,81,178_மனித_உரிமை_ஐநா_மீறல்


In [11]:
# Get the words in a topic

topic_model.get_topic(0)


[('பலி', 0.05313793400794363),
 ('மோதி', 0.052433766057955564),
 ('லாரி', 0.039035603018047545),
 ('பரிதாப', 0.03259167853474521),
 ('கார்', 0.029732657557268012),
 ('படுகாயம்', 0.028991250301783872),
 ('பஸ்', 0.028243137731692992),
 ('விபத்து', 0.02728168858405848),
 ('வேன்', 0.02372532047915434),
 ('சாவு', 0.02309325474333672)]

In [12]:
topic_model.get_representative_docs(0)


['லாரி மீது வேன் மோதி பேர் பரிதாப பலி',
 'லாரி மீது கார் மோதி பேர் பரிதாப சாவு',
 'பைக் மீது பஸ் மோதி பேர் பரிதாப பலி']

In [13]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = topic_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(topic_model.get_topics())-1)]


In [14]:

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_npmi')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.0764972852765826

In [15]:
topic_model.visualize_topics()